# Shamir Secret Sharing Voting

In this notebook,
we explore a voting protocol which uses Shamir's Secret Sharing (SSS)
to aggregate votes,
without individual votes being discoverable.
This protocol is best suited to a voting session with well-defined,
non-cooperating parties,
such as political parties in a general election.
However,
we only need _K_ vote counters to act honestly
in order to decrypt the results,
where _K_ is a variable to be decided on by the vote controller.

*THIS IS A POC. SSI/SECURE COMMUNICATION IS NOT IN PLACE.*

For this POC,
we shall perform a categorical vote on "Best PriCon workshop".

In [ ]:
import random
import uuid
from decimal import Decimal

import numpy as np

## Define Voting Session

We create a helper class to manage voting options

In [ ]:
class Vote:
    def __init__(self, *options):
        vote_values = {}
        for i, option in enumerate(options):
            vote_values[option] = i

        self._vote_values = vote_values
        self._vote_classes = {v: k for k, v in vote_values.items()}

    def __len__(self):
        return len(self._vote_values)

    def __getitem__(self, i):
        return self._vote_classes[i]

    def get_vote_class(self, vote_value):
        return self._vote_values[vote_value]

    def get_vote_value(self, vote_class):
        return self._vote_classes[vote_class]

In [ ]:
pricon_vote = Vote("PryVote", "PyDP", "PyVertical")

## Define roles

We create simple _Voter_ and _Party_ classes.

Voter:
- Given a unique ID
- Can store a personal vote value
- Can send vote and ID to parties

Party:
- Can aggregate votes
- Can share vote aggregates amongst other parties

In [ ]:
class Voter:
    def __init__(self, Q: int, n_counters: int = 2) -> None:
        self._id = uuid.uuid4()
        self._Q = Q
        self._vote_shares = None

    def _get_polynomial(self, x, coeff): 
        return sum([x**(len(coeff)-i-1) * coeff[i] for i in range(len(coeff))]) 
   
    def _get_coeffs(self, t, secret):
        coeff = [random.randint(0, self._Q) for _ in range(t-1)] 
        coeff.append(secret) 

        return coeff 

    def _encrypt_vote(self, n, m, secret): 
        cfs = self._get_coeffs(m,secret) 
        shares = [] 

        for i in range(1,n+1): 
            shares.append((i, self._get_polynomial(i,cfs)))

        return shares 

    def update_vote(self, vote_value: str) -> None:
        try:
            vote_class = pricon_vote.get_vote_class(vote_value)

            onehot_votes = [0] * len(pricon_vote)
            onehot_votes[vote_class] = 1
            
            vote_shares = []

            for binary_vote in onehot_votes:
                vote_shares.append(self._encrypt_vote(3, 2, binary_vote))

            self._vote_shares = list(map(list, zip(*vote_shares)))
        except KeyError:
            self._vote_shares = None

    def send_vote(self, parties) -> None:
        if self._vote_shares is None:
            print(f"{self._id} has not set a vote")
            return

        assert len(self._vote_shares) == len(parties)

        for vote_share, party in zip(self._vote_shares, parties):
            party.receive_vote(self._id, vote_share)

In [ ]:
class Party:
    def __init__(self, name: str, Q: int, vote_counter_number: int) -> None:
        self._name = name
        self._Q = Q
        self._ids = []
        self._vote_sum = [(vote_counter_number, 0)] * len(pricon_vote)  # init to zero for each vote count

    def receive_vote(self, voter_id: uuid.UUID, votes) -> None:
        if voter_id not in self._ids:
            print(f"{self._name}: Adding vote for {voter_id}")
            self._ids.append(voter_id)

            for i in range(len(votes)):
                vote_counter_number, vote_sum = self._vote_sum[i]
                assert votes[i][0] == vote_counter_number  # x point should be constant
                self._vote_sum[i] = (vote_counter_number, vote_sum + votes[i][1])
        else:
            print(f"{self._name}: {voter_id} has already voted")

## Define Q

In [ ]:
Q = 1234567891011

## Create Voters and Vote counters

5 voters, 3 vote counters (red, blue, yellow)

In [ ]:
red = Party("red", Q, 1)
blue = Party("blue", Q, 2)
yellow = Party("yellow", Q, 3)

MAX_VOTERS = 3  # in practice, determined by credential issuance

In [ ]:
alice = Voter(Q)
bob = Voter(Q)
charlie = Voter(Q)
dan = Voter(Q)
eve = Voter(Q)
fran = Voter(Q)
greg = Voter(Q)

## Vote!

Each voter splits their votes into 3 shares
and shares only one share with each of the three vote counters.
No vote counter can work out what the vote was,
given only a single share -
you need all to somehow get hold of all three to be able to reverse the encryption!
However,
the shares a vote counter receives can be combined.

<img src="../images/smpv-1.png" alt="Voting Protocol" width="900"/>

In [ ]:
alice.update_vote("PryVote")
bob.update_vote("PryVote")
charlie.update_vote("PyDP")
dan.update_vote("PryVote")
eve.update_vote("PyDP")
fran.update_vote("PyVertical")
greg.update_vote("PyVertical")

In [ ]:
for voter in [alice, bob, charlie, dan, eve, fran, greg]:
    voter.send_vote([red, blue, yellow])

What do the votes look like to a vote counter?

In [ ]:
red._vote_sum, blue._vote_sum, yellow._vote_sum

## Decrypt the Vote

When the voting is done,
the vote counters share the _sums_
of the secret vote shares
they received.
Together,
the sums of shares can be decrypted
to find a _result_ -
no individual vote can be decrypted.

<img src="../images/smpv-2.png" alt="Voting Protocol" width="900"/>

## Decrypt using pairs of vote counters

In [ ]:
vote_counters = {"red": red,
                 "blue": blue,
                 "yellow": yellow}

for counter_1_idx, (vc_1_name, vc_1) in enumerate(vote_counters.items()):
    for counter_2_idx, (vc_2_name, vc_2) in enumerate(vote_counters.items()):
        if counter_2_idx <= counter_1_idx:
            continue

        print(f"\n\nDecrypting results using {vc_1_name} and {vc_2_name}...")

        vote_counts = []
        vote_values = []

        for i in range(len(pricon_vote)):
            vote_value = pricon_vote[i]

            first_counter_shares = vc_1._vote_sum[i]
            second_counter_shares = vc_2._vote_sum[i]
            print(second_counter_shares)

            l0 = -second_counter_shares[0] / (first_counter_shares[0] - second_counter_shares[0])
            l1 = -first_counter_shares[0] / (second_counter_shares[0] - first_counter_shares[0])

            result = l0 * first_counter_shares[1] + l1 * second_counter_shares[1]
            if result > Q:
                result = result % Q
            result = int(result)

            vote_values.append(vote_value)
            vote_counts.append(result)

        legitimate_score = np.max(vote_counts) <= MAX_VOTERS
        winning_index = np.argmax(vote_counts)
        winner = pricon_vote.get_vote_value(winning_index)

        if legitimate_score:
            print(f"Vote counts are {vote_counts}\n{winner} is the winner!")
        else:
            print(f"Vote counts incorrect. Count was {np.max(vote_counts)}, but max voters is {MAX_VOTERS}")

We see that the correct result is produced for each pair of vote counters.
In practice, this means that the voting system can be made robust against malicious vote counters.

But what if yellow acted maliciously and changed the secret share it owned?

In [ ]:
yellow._vote_sum[1] = (3, 2_000_000)

In [ ]:
vote_counters = {"red": red,
                 "blue": blue,
                 "yellow": yellow}

for counter_1_idx, (vc_1_name, vc_1) in enumerate(vote_counters.items()):
    for counter_2_idx, (vc_2_name, vc_2) in enumerate(vote_counters.items()):
        if counter_2_idx <= counter_1_idx:
            continue

        print(f"\n\nDecrypting results using {vc_1_name} and {vc_2_name}...")

        vote_counts = []
        vote_values = []

        for i in range(len(pricon_vote)):
            vote_value = pricon_vote[i]

            first_counter_shares = vc_1._vote_sum[i]
            second_counter_shares = vc_2._vote_sum[i]

            l0 = -second_counter_shares[0] / (first_counter_shares[0] - second_counter_shares[0])
            l1 = -first_counter_shares[0] / (second_counter_shares[0] - first_counter_shares[0])

            result = l0 * first_counter_shares[1] + l1 * second_counter_shares[1]
            if result > Q:
                result = result % Q
            result = int(result)

            vote_values.append(vote_value)
            vote_counts.append(result)

        legitimate_score = np.max(vote_counts) <= MAX_VOTERS
        winning_index = np.argmax(vote_counts)
        winner = pricon_vote.get_vote_value(winning_index)

        if legitimate_score:
            print(f"Vote counts are {vote_counts}\n{winner} is the winner!")
        else:
            print(f"Vote counts incorrect. Count was {np.max(vote_counts)}, but max voters is {MAX_VOTERS}")

---

## Evaluation of the protocol